In [1]:
import pandas as pd
import os
from unicodedata import normalize
import re
import math as m
import datetime as dt
import numpy as np

In [3]:
path = 'path1'

In [4]:
def clean_datalist(data):
    #Se toman todos los tweets y se pasan a lista
    #data = dft.text.values.tolist()
    #Remover e-mails y menciones de otras cuentas de twitter con @
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    #Remover caracteres de nueva línea
    data = [re.sub('\s+', ' ', sent) for sent in data]
    #Remover comillas simples
    data = [re.sub("\'", "", sent) for sent in data]
    #Remover URLs hhtp y https
    data = [re.sub(r"https://\S+", "", sent) for sent in data]
    data = [re.sub(r"http://\S+", "", sent) for sent in data]
    #Remover datos entre corchetes y parentesis por ejemplo [VIDEO]
    data = [re.sub("[\(\[].*?[\)\]]", "", sent) for sent in data]
    #Remove Minusculas
    data = [sent.lower() for sent in data]
    #Maneja ñ
    data = [sent.replace('ñ','n') for sent in data]
    #Remover puntuaciones
    data = [re.sub('[^\w\s]', "", sent) for sent in data]
    #Remover acentos
    data = [normalize('NFC',re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", sent), 0, re.I)) for sent in data]
    #Remover espacios de inicio y fin
    data = [sent.lstrip(' ').rstrip(' ') for sent in data]
    
    return data

# Read Cases

In [6]:
dfc = pd.read_csv(path+'cases.csv',encoding='utf-8')
dfc.region = clean_datalist(dfc.region.values.tolist())
dfc

,date,region,new_cases,new_deaths,cum_cases,cum_deaths
0,2020-03-03,antofagasta,0,0,0.0,0
1,2020-03-04,antofagasta,0,0,0.0,0
2,2020-03-05,antofagasta,0,0,0.0,0
3,2020-03-06,antofagasta,0,0,0.0,0
4,2020-03-07,antofagasta,0,0,0.0,0
...,...,...,...,...,...,...
8954,2021-08-07,valparaiso,83,3310,120203.0,637113
8955,2021-08-08,valparaiso,76,3315,120279.0,640428
8956,2021-08-09,valparaiso,65,3325,120344.0,643753
8957,2021-08-10,valparaiso,35,3324,120379.0,647077


# Get Predictions

In [7]:
dfp = pd.read_csv(path+'region_predictions_2020_Jul.csv',encoding='utf-8')
dfp.region = clean_datalist(dfp.region.values.tolist())
dfp

,date,region,estimate,lower,upper
0,2020-06-01,coquimbo,0.956297,0.816712,1.000000
1,2020-06-02,coquimbo,0.957190,0.821517,1.000000
2,2020-06-03,coquimbo,0.958024,0.826473,1.000000
3,2020-06-04,coquimbo,0.958803,0.830766,1.000000
4,2020-06-05,coquimbo,0.959531,0.834351,1.000000
...,...,...,...,...,...
923,2020-06-27,maule,0.752626,0.617109,0.881460
924,2020-06-28,maule,0.731119,0.598061,0.859811
925,2020-06-29,maule,0.707695,0.577460,0.836593
926,2020-06-30,maule,0.682458,0.555897,0.812354


In [8]:
dfex = dfp.merge(dfc,how='left',left_on=['date','region'],right_on=['date','region'])

dfex['deaths'] = 0
dfe = pd.DataFrame(columns=dfex.columns)
for i in dfex.region.unique():
    df_aux = dfex[dfex.region==i].copy()
    df_aux['deaths'] = df_aux.new_deaths.diff(1)
    df_aux.cum_deaths = df_aux.new_deaths
    df_aux.new_deaths = df_aux['deaths'].apply(lambda x: 0 if x<0 or m.isnan(x) else x)
    dfe = dfe.append(df_aux)
    #new_deaths[::-1].cumsum() 
    #break
dfe = dfe[dfe.columns[:-1]]
dfe

,date,region,estimate,lower,upper,new_cases,new_deaths,cum_cases,cum_deaths
0,2020-06-01,coquimbo,0.956297,0.816712,1.000000,40,0.0,668.0,2
1,2020-06-02,coquimbo,0.957190,0.821517,1.000000,19,0.0,687.0,2
2,2020-06-03,coquimbo,0.958024,0.826473,1.000000,31,2.0,718.0,4
3,2020-06-04,coquimbo,0.958803,0.830766,1.000000,54,0.0,772.0,4
4,2020-06-05,coquimbo,0.959531,0.834351,1.000000,60,1.0,832.0,5
...,...,...,...,...,...,...,...,...,...
923,2020-06-27,maule,0.752626,0.617109,0.881460,169,2.0,5077.0,41
924,2020-06-28,maule,0.731119,0.598061,0.859811,140,7.0,5217.0,48
925,2020-06-29,maule,0.707695,0.577460,0.836593,141,0.0,5358.0,48
926,2020-06-30,maule,0.682458,0.555897,0.812354,86,0.0,5444.0,48


# Dictionaries

In [10]:
dict_regiones = {
    'tarapaca' : 1,
    'antofagasta' : 2,
    'atacama' : 3,
    'coquimbo' : 4,
    'valparaiso' : 5,
    'ohiggins' : 6,
    'maule' : 7,
    'biobio' : 8,
    'araucania' : 9,
    'los lagos' : 10,
    'aysen' : 11,
    'magallanes' : 12,
    'metropolitana' : 13,
    'los rios' : 14,
    'arica y parinacota' : 15,
    'nuble' : 16
}

dict_regiones2 = {
    1:'tarapaca',
    2:'antofagasta',
    3:'atacama',
    4:'coquimbo',
    5:'valparaiso',
    6:'ohiggins',
    7:'maule',
    8:'biobio',
    9:'araucania',
    10:'los lagos',
    11:'aysen',
    12:'magallanes',
    13:'metropolitana',
    14:'los rios',
    15:'arica y parinacota',
    16:'nuble'
}



dfe['cod_reg'] = dfe.region.map(dict_regiones)
dfe['cod_reg'] = dfe['cod_reg'].apply(lambda x: int(x))
dfe

,date,region,estimate,lower,upper,new_cases,new_deaths,cum_cases,cum_deaths,cod_reg
0,2020-06-01,coquimbo,0.956297,0.816712,1.000000,40,0.0,668.0,2,4
1,2020-06-02,coquimbo,0.957190,0.821517,1.000000,19,0.0,687.0,2,4
2,2020-06-03,coquimbo,0.958024,0.826473,1.000000,31,2.0,718.0,4,4
3,2020-06-04,coquimbo,0.958803,0.830766,1.000000,54,0.0,772.0,4,4
4,2020-06-05,coquimbo,0.959531,0.834351,1.000000,60,1.0,832.0,5,4
...,...,...,...,...,...,...,...,...,...,...
923,2020-06-27,maule,0.752626,0.617109,0.881460,169,2.0,5077.0,41,7
924,2020-06-28,maule,0.731119,0.598061,0.859811,140,7.0,5217.0,48,7
925,2020-06-29,maule,0.707695,0.577460,0.836593,141,0.0,5358.0,48,7
926,2020-06-30,maule,0.682458,0.555897,0.812354,86,0.0,5444.0,48,7


# Population

In [11]:
dfpobla = pd.read_excel(path+'Poblacion Region.xlsx')
dfpobla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   nombre_region  16 non-null     object
 1   cod_region     16 non-null     int64 
 2   Población      16 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 512.0+ bytes


In [12]:
dfur = dfe.merge(dfpobla[['cod_region','Población']],how='left',left_on='cod_reg',right_on='cod_region')
dfur = dfur[['date','region','cod_region','estimate','lower','upper','new_cases','new_deaths','Población']]
dfur = dfur.rename(columns={'new_cases':'cases','new_deaths':'deaths','Población':'population'})
dfur.to_csv(path+'generated/'+'chile_under_reporting_data.csv')
dfur

,date,region,cod_region,estimate,lower,upper,cases,deaths,population
0,2020-06-01,coquimbo,4,0.956297,0.816712,1.000000,40,0.0,757586
1,2020-06-02,coquimbo,4,0.957190,0.821517,1.000000,19,0.0,757586
2,2020-06-03,coquimbo,4,0.958024,0.826473,1.000000,31,2.0,757586
3,2020-06-04,coquimbo,4,0.958803,0.830766,1.000000,54,0.0,757586
4,2020-06-05,coquimbo,4,0.959531,0.834351,1.000000,60,1.0,757586
...,...,...,...,...,...,...,...,...,...
923,2020-06-27,maule,7,0.752626,0.617109,0.881460,169,2.0,1044950
924,2020-06-28,maule,7,0.731119,0.598061,0.859811,140,7.0,1044950
925,2020-06-29,maule,7,0.707695,0.577460,0.836593,141,0.0,1044950
926,2020-06-30,maule,7,0.682458,0.555897,0.812354,86,0.0,1044950


In [13]:
dfed = dfc.copy()
dfed['cod_reg'] = dfed.region.map(dict_regiones)
dfed = dfed.dropna().reset_index(drop=True)
dfed['cod_reg'] = dfed['cod_reg'].apply(lambda x: int(x))
dfed = dfed.merge(dfpobla,how='left',left_on='cod_reg',right_on='cod_region')
dfed = dfed.rename(columns={'new_cases':'cases','new_deaths':'deaths','Población':'population'})
dfed = dfed[['date','region','cod_region','cases','deaths','population']]
dfed.to_csv(path+'generated/'+'chile_ecdc_data.csv')
dfed

,date,region,cod_region,cases,deaths,population
0,2020-03-03,antofagasta,2,0,0,607534
1,2020-03-04,antofagasta,2,0,0,607534
2,2020-03-05,antofagasta,2,0,0,607534
3,2020-03-06,antofagasta,2,0,0,607534
4,2020-03-07,antofagasta,2,0,0,607534
...,...,...,...,...,...,...
8427,2021-08-07,valparaiso,5,83,3310,1815902
8428,2021-08-08,valparaiso,5,76,3315,1815902
8429,2021-08-09,valparaiso,5,65,3325,1815902
8430,2021-08-10,valparaiso,5,35,3324,1815902


# Travel info

In [ ]:
df_viajes = pd.read_csv(path+'escenarios_viajes_regionales_new2.csv',sep=',')
df_viajes.date = df_viajes.date.apply(lambda x: dt.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m-%d'))
df_viajes = df_viajes[df_viajes.region_origin_code!=df_viajes.region_destination_code].sort_values(['date','region_origin_code','region_destination_code']).reset_index(drop=True)
df_viajes

# Scenarios Info

In [21]:
# Escenario 1: 2Mar-15Mar
# Escenario 2: 30Mar - 12Apr // 
# Escenario 3: 18May - 31May // 
# Inicio total 17 Jun
import datetime as dt

In [ ]:
sce_1 = dt.datetime(2020,3,2)
sce_2 = dt.datetime(2020,3,30)
sce_3 = dt.datetime(2020,5,18)
analysis_date = dt.datetime(2020,6,17)


df_travel_scenarios_sim = pd.DataFrame(columns=['date_cases_y_ascertainments','region_origin_code','region_destination_code','travel_scenario_1','date_scenario_1',\
                                            'travel_scenario_2','date_scenario_2','travel_scenario_3','date_scenario_3'])

for i in range(0,14):
    df_aux = pd.DataFrame(l,columns=['region_origin_code','region_destination_code'])

    #Escenario 1
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_1+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_1'})
    df_aux['date_scenario_1'] = (sce_1+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Escenario 2
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_2+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_2'})
    df_aux['date_scenario_2'] = (sce_2+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Escenario 3
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_3+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_3'})
    df_aux['date_scenario_3'] = (sce_3+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Fecha casos
    df_aux['date_cases_y_ascertainments'] = (analysis_date+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Ajustar df
    df_aux = df_aux[['date_cases_y_ascertainments','region_origin_code','region_destination_code','travel_scenario_1','date_scenario_1',\
                                            'travel_scenario_2','date_scenario_2','travel_scenario_3','date_scenario_3']]
    #Appendear a df_travel_scenarios
    df_travel_scenarios_sim = df_travel_scenarios_sim.append(df_aux)

df_travel_scenarios_sim.to_csv(path+'generated/'+'all_travel_scenarios_sim.csv')
df_travel_scenarios_sim

In [ ]:
sce_2 = dt.datetime(2020,5,8)
sce_3 = dt.datetime(2020,5,23)
sce_sim = dt.datetime(2020,3,1)
analysis_date = dt.datetime(2020,6,17)


df_travel_scenarios = pd.DataFrame(columns=['region_origin_code','region_destination_code', 'travel_scenario_2','date_scenario_2',\
                                           'travel_scenario_3','date_scenario_3','travel_scenario_sim','date_scenario_sim'])

for i in range(0,10):
    df_aux = pd.DataFrame(l,columns=['region_origin_code','region_destination_code'])

    #Escenario 1
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_2+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_2'})
    df_aux['date_scenario_2'] = (sce_2+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Escenario 2
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_3+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_3'})
    df_aux['date_scenario_3'] = (sce_3+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Escenario Sim
    df_aux = df_aux.merge(df_viajes[['region_origin_code','region_destination_code','cnt']][df_viajes.date==(sce_sim+dt.timedelta(days=i)).strftime("%Y-%m-%d")]\
                         ,how='left',left_on=['region_origin_code','region_destination_code'],right_on=['region_origin_code','region_destination_code']).fillna(0)
    df_aux = df_aux.rename(columns={'cnt':'travel_scenario_sim'})
    df_aux['date_scenario_sim'] = (sce_sim+dt.timedelta(days=i)).strftime("%Y-%m-%d")
    #Ajustar df
    df_aux = df_aux[['region_origin_code','region_destination_code', 'travel_scenario_2','date_scenario_2',\
                                           'travel_scenario_3','date_scenario_3','travel_scenario_sim','date_scenario_sim']]
    #Appendear a df_travel_scenarios
    df_travel_scenarios = df_travel_scenarios.append(df_aux)

df_travel_scenarios.to_csv(path+'generated/'+'all_travel_scenarios_definidos.csv')
df_travel_scenarios

# R Values

In [40]:
files_r = os.listdir(path_R)

dfr = pd.read_csv(path_R+files_r[0],index_col=0)
dfr['region'] = clean_datalist([files_r[0].split('_')[0]])[0]

for i,j in enumerate(files_r[1:]):
    df_aux = pd.read_csv(path_R+j,index_col=0)
    df_aux['region'] = clean_datalist([j.split('_')[0]])[0]
    dfr = dfr.append(df_aux)

dfr = dfr.reset_index(drop=True)
dfr['cod_reg'] = dfr.region.map(dict_regiones)
dfr['cod_reg'] = dfr['cod_reg'].apply(lambda x: 0 if np.isnan(x) else int(x))
dfr

,t_start,t_end,Mean(R),Std(R),Quantile.0.025(R),Quantile.0.05(R),Quantile.0.25(R),Median(R),Quantile.0.75(R),Quantile.0.95(R),Quantile.0.975(R),fecha_casos,region,cod_reg
0,2,8,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-03,antofagasta,2
1,3,9,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-04,antofagasta,2
2,4,10,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-05,antofagasta,2
3,5,11,5.000000,5.000000,0.126589,0.256466,1.438410,3.465736,6.931472,14.978661,18.444397,2020-03-06,antofagasta,2
4,6,12,15.000000,8.660254,3.093361,4.088457,8.636497,13.370302,19.602010,31.478968,36.123438,2020-03-07,antofagasta,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9005,527,533,0.835557,0.041935,0.755376,0.767796,0.806902,0.834855,0.863446,0.905710,0.919723,2021-08-10,biobio,8
9006,528,534,0.865269,0.043318,0.782440,0.795272,0.835671,0.864546,0.894079,0.937732,0.952205,2021-08-11,biobio,8
9007,529,535,0.943123,0.045748,0.855578,0.869157,0.911877,0.942384,0.973564,1.019613,1.034871,2021-08-12,biobio,8
9008,530,536,0.977558,0.046763,0.888041,0.901932,0.945623,0.976812,1.008679,1.055726,1.071311,2021-08-13,biobio,8


# S2 and S3

In [ ]:
dfic_s2 = pd.read_csv(path+'imported_cases_and_incidence_together_s2.csv')

dfic_s2 = dfic_s2.merge(dfr_s2[['Median(R)','Quantile.0.025(R)','Quantile.0.975(R)','cod_reg']],how='left',left_on='region_destination_code',right_on='cod_reg')
dfic_s2 = dfic_s2.rename(columns={'Median(R)':'r_mid', 'Quantile.0.025(R)':'r_low', 'Quantile.0.975(R)':'r_high'})
dfic_s2.drop(columns=['cod_reg'],inplace=True)

dfic_s2['expected_imported_cases_scenario_2'] = dfic_s2.expected_imported_cases_scenario_2_mid.round(2).astype(str)+' ('+dfic_s2.expected_imported_cases_scenario_2_low.round(2).astype(str)+', '+\
                                        dfic_s2.expected_imported_cases_scenario_2_high.round(2).astype(str)+')'

dfic_s2['expected_imported_cases_scenario_2sim'] = dfic_s2.expected_imported_cases_scenario_2sim_mid.round(2).astype(str)+' ('+dfic_s2.expected_imported_cases_scenario_2sim_low.round(2).astype(str)+', '+\
                                        dfic_s2.expected_imported_cases_scenario_2sim_high.round(2).astype(str)+')'

dfic_s2['local_cases_scenario_2'] = dfic_s2.new_cases_adjusted_mean_mid.round(2).astype(str)+' ('+dfic_s2.new_cases_adjusted_mean_low.round(2).astype(str)+', '+\
                                        dfic_s2.new_cases_adjusted_mean_high.round(2).astype(str)+')'

dfic_s2['risk_rating_scenario_2'] = (dfic_s2.expected_imported_cases_scenario_2_mid/dfic_s2.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s2.expected_imported_cases_scenario_2_low/dfic_s2.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s2.expected_imported_cases_scenario_2_high/dfic_s2.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s2['risk_rating_scenario_2sim'] = (dfic_s2.expected_imported_cases_scenario_2sim_mid/dfic_s2.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s2.expected_imported_cases_scenario_2sim_low/dfic_s2.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s2.expected_imported_cases_scenario_2sim_high/dfic_s2.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s2['rt_estimate_scenario_2'] = dfic_s2.r_mid.round(2).astype(str)+' ('+dfic_s2.r_low.round(2).astype(str)+', '+\
                                        dfic_s2.r_high.round(2).astype(str)+')'

dfic_s2['region'] = dfic_s2.region_destination_code.map(dict_regiones2)
dfic_s2 = dfic_s2.rename(columns={'region_destination_code':'cod_reg'})

dfic_s2.to_csv(path+'results_s2.csv')

dfic_s2[['cod_reg','region','expected_imported_cases_scenario_2','risk_rating_scenario_2','local_cases_scenario_2','rt_estimate_scenario_2',
         'risk_rating_scenario_2sim','expected_imported_cases_scenario_2sim']]

In [ ]:
dfic_s3 = pd.read_csv(path+'imported_cases_and_incidence_together_s3.csv')

dfic_s3 = dfic_s3.merge(dfr_s3[['Median(R)','Quantile.0.025(R)','Quantile.0.975(R)','cod_reg']],how='left',left_on='region_destination_code',right_on='cod_reg')
dfic_s3 = dfic_s3.rename(columns={'Median(R)':'r_mid', 'Quantile.0.025(R)':'r_low', 'Quantile.0.975(R)':'r_high'})
dfic_s3.drop(columns=['cod_reg'],inplace=True)

dfic_s3['expected_imported_cases_scenario_3'] = dfic_s3.expected_imported_cases_scenario_3_mid.round(2).astype(str)+' ('+dfic_s3.expected_imported_cases_scenario_3_low.round(2).astype(str)+', '+\
                                        dfic_s3.expected_imported_cases_scenario_3_high.round(2).astype(str)+')'

dfic_s3['expected_imported_cases_scenario_3sim'] = dfic_s3.expected_imported_cases_scenario_3sim_mid.round(2).astype(str)+' ('+dfic_s3.expected_imported_cases_scenario_3sim_low.round(2).astype(str)+', '+\
                                        dfic_s3.expected_imported_cases_scenario_3sim_high.round(2).astype(str)+')'

dfic_s3['local_cases_scenario_3'] = dfic_s3.new_cases_adjusted_mean_mid.round(2).astype(str)+' ('+dfic_s3.new_cases_adjusted_mean_low.round(2).astype(str)+', '+\
                                        dfic_s3.new_cases_adjusted_mean_high.round(2).astype(str)+')'

dfic_s3['risk_rating_scenario_3'] = (dfic_s3.expected_imported_cases_scenario_3_mid/dfic_s3.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s3.expected_imported_cases_scenario_3_low/dfic_s3.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s3.expected_imported_cases_scenario_3_high/dfic_s3.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s3['risk_rating_scenario_3sim'] = (dfic_s3.expected_imported_cases_scenario_3sim_mid/dfic_s3.new_cases_adjusted_mean_mid).round(3).astype(str)+' ('+\
                                        (dfic_s3.expected_imported_cases_scenario_3sim_low/dfic_s3.new_cases_adjusted_mean_low).round(3).astype(str)+', '+\
                                        (dfic_s3.expected_imported_cases_scenario_3sim_high/dfic_s3.new_cases_adjusted_mean_high).round(3).astype(str)+')'

dfic_s3['rt_estimate_scenario_3'] = dfic_s3.r_mid.round(2).astype(str)+' ('+dfic_s3.r_low.round(2).astype(str)+', '+\
                                        dfic_s3.r_high.round(2).astype(str)+')'

dfic_s3['region'] = dfic_s3.region_destination_code.map(dict_regiones2)
dfic_s3 = dfic_s3.rename(columns={'region_destination_code':'cod_reg'})

dfic_s3.to_csv(path+'results_s3.csv')

dfic_s3[['cod_reg','region','expected_imported_cases_scenario_3','risk_rating_scenario_3','local_cases_scenario_3','rt_estimate_scenario_3',
         'risk_rating_scenario_3sim','expected_imported_cases_scenario_3sim']]